 Create Source System database table

In [1]:
CREATE TABLE SourceSystem (
		source_system_id int UNIQUE PRIMARY KEY,
		system_name varchar(32) NULL,
		system_version varchar(16) NULL,
        system_type varchar(32) NULL,
		external_schema varchar(16) NULL,
		is_production bit NOT NULL
);


Commands completed successfully

Total execution time: 00:00:00.013

Create Source System table

In [ ]:
DROP TABLE IF EXISTS temp_source_system;

CREATE TEMPORARY TABLE temp_source_system (
    source_system_id int NOT NULL,
    system_name varchar(32) NULL,
    system_version varchar(16) NULL,
    system_type varchar(32) NULL,
    external_schema varchar(16) NULL,
    is_production bit NOT NULL
);

INSERT INTO temp_source_system (
	  source_system_id
	, system_name
	, system_version
	, system_type
	, external_schema
	, is_production
)
VALUES
	(0, 'Unknown', NULL, NULL, NULL, '0'),
	(1, 'Astea', '5.5', 'ERP', 'astea5', '0'),
	(2, 'Astea', '14', 'ERP', 'astea14', '1'),
	(3, 'Salesforce', NULL, 'CRM', 'sf', '1');

-- Update if existing
UPDATE public.SourceSystem s
SET system_name = t.system_name
FROM temp_source_system t
WHERE s.source_system_id = t.source_system_id AND s.system_name <> t.system_name;

-- Insert if missing 
INSERT INTO public.SourceSystem (
  	  source_system_id
	, system_name
	, system_version
	, system_type
	, external_schema
	, is_production  
)
SELECT
  	  t.source_system_id
	, t.system_name
	, t.system_version
	, t.system_type
	, t.external_schema
	, t.is_production  
    --, s.source_system_id
FROM temp_source_system t
    LEFT OUTER JOIN public.SourceSystem s ON t.source_system_id = s.source_system_id
WHERE s.source_system_id IS NULL;

Generate schemas that will be used by the data warehouse.

1.  dwh - This schema will contain tables and stored procedures related to the core data warehouse
2.  master - This schema will contain tables and stored procedures used for holding normalized master data
3.  olap - This schema will contain views to expose data from the dwh and master schemas to Analysis Services
4.  stage - This schema will contain tables used for storing data that needs to be staged before or during the ETL process.
5.  Any schemas listed in the external\_schema column in the \[dbo\].\[SourceSystem\] table.

In [ ]:
-- Set up temp table
DROP TABLE IF EXISTS temp_schemas;

CREATE TEMPORARY TABLE temp_schemas
(
	Id INT GENERATED ALWAYS AS IDENTITY,
    SchemaName VARCHAR(32)
);

-- Insert standard schemas
INSERT INTO temp_schemas (SchemaName) VALUES ('dwh'); --Data Warehouse Schema
INSERT INTO temp_schemas (SchemaName) VALUES ('master'); --Master table Schema
INSERT INTO temp_schemas (SchemaName) VALUES ('olap'); --SSAS View Schema
INSERT INTO temp_schemas (SchemaName) VALUES ('stage'); --Staging table Schema

-- Insert schemas from SourceSystem table
INSERT INTO temp_schemas (SchemaName)
SELECT DISTINCT external_schema
FROM public.SourceSystem
WHERE external_schema NOT IN (
    SELECT SchemaName FROM temp_schemas
);

-- Create schemas
DO $$
DECLARE
    r RECORD;
BEGIN
    -- Loop through all records in temp_schemas
    FOR r IN SELECT SchemaName FROM temp_schemas
    LOOP
        -- Create schema
        EXECUTE format('CREATE SCHEMA %s', r.SchemaName);

    END LOOP;
END; $$

Create standard stored procedures for importing data.

In [ ]:
DO $$
DECLARE
    r RECORD;
BEGIN
    -- Loop through all records in temp_schemas
    FOR r IN SELECT DISTINCT system_name FROM public.SourceSystem WHERE system_name <> 'Unknown'
    LOOP
        -- Create schema
        EXECUTE format('CREATE PROCEDURE %s() 
        LANGUAGE SQL AS $BODY$  
            --RAISE NOTICE "Insert logic here";
        $BODY$', r.system_name);

    END LOOP;
END; $$

Create store procedure for creating a master date table.

In [ ]:
CREATE OR REPLACE FUNCTION create_calendar_table(startdate DATE, enddate DATE, fiscalstartdate DATE) 
RETURNS INTEGER AS $$ 

DECLARE 
    daydiff INTERVAL;
    record_count INTEGER;
BEGIN
    -- Calculate date diff
    daydiff:=make_interval(days => startdate - fiscalstartdate);
    
    -- Create Holiday Table if doesn't exist
    CREATE TABLE IF NOT EXISTS master.Holiday (
        holiday_id int PRIMARY KEY,
        holiday_key char(18) NULL,
        actual_date date NOT NULL,
        observed_date date NOT NULL,
        description varchar(32) NULL,
        dwh_created_date timestamp NOT NULL,
        dwh_modified_date timestamp NULL
    );

    -- Create the table
    CREATE TABLE IF NOT EXISTS master.Calendar (
        full_date date NOT NULL,
        day_of_week int NOT NULL DEFAULT 0,
        day_of_week_name varchar(10) NOT NULL DEFAULT '',
        day_of_week_abbrev varchar(5) NOT NULL DEFAULT '',
        day_of_month int NOT NULL DEFAULT 0,
        day_of_quarter int NOT NULL DEFAULT 0,
        day_of_year int NOT NULL DEFAULT 0,
        week_of_year int NOT NULL DEFAULT 0,
        month_of_year int NOT NULL DEFAULT 0,
        month_name varchar(10) NOT NULL DEFAULT '',
        month_name_abbrev varchar(5) NOT NULL DEFAULT '',
        quarter_of_year int NOT NULL DEFAULT 0,
        year int NOT NULL DEFAULT 0,
        fiscal_day_of_month int NOT NULL DEFAULT 0,
        fiscal_day_of_quarter int NOT NULL DEFAULT 0,
        fiscal_day_of_year int NOT NULL DEFAULT 0,
        fiscal_week_of_year int NOT NULL DEFAULT 0,
        fiscal_month_of_year int NOT NULL DEFAULT 0,
        fiscal_quarter_of_year int NOT NULL DEFAULT 0,
        fiscal_year int NOT NULL DEFAULT 0,
        is_business_day BOOLEAN NOT NULL DEFAULT FALSE,
        is_weekday BOOLEAN NOT NULL DEFAULT FALSE,
        is_holiday BOOLEAN NOT NULL DEFAULT FALSE,
        YYYYMM char(6) NOT NULL DEFAULT '',
        YYYYMMDD char(8) NOT NULL DEFAULT '',
        YYYYQQ char(7) NOT NULL DEFAULT '',
        YYYYFQ char(7) NOT NULL DEFAULT ''
    );

    -- Make sure table is clean
    TRUNCATE TABLE master.Calendar;

    -- Insert records into table
    INSERT INTO master.Calendar (
        full_date
        , day_of_week
        , day_of_week_name
        , day_of_week_abbrev
        , day_of_month
        , day_of_quarter
        , day_of_year
        , week_of_year
        , month_of_year
        , month_name
        , month_name_abbrev
        , quarter_of_year
        , year
        , fiscal_day_of_month
        , fiscal_day_of_quarter
        , fiscal_day_of_year
        , fiscal_week_of_year
        , fiscal_month_of_year
        , fiscal_quarter_of_year
        , fiscal_year
        , is_business_day
        , is_weekday
        , is_holiday
        , YYYYMM
        , YYYYMMDD
        , YYYYQQ
        , YYYYFQ
    )
    SELECT
        CAST(generate_series AS date) AS full_date
        , EXTRACT(DOW FROM generate_series)::INTEGER AS day_of_week
        , to_char(generate_series, 'Day') AS day_of_week_name
        , to_char(generate_series, 'Dy') AS day_of_week_abbrev
        , EXTRACT(DAY FROM generate_series)::INTEGER AS day_of_month
        , DATE_PART('day', generate_series - date_trunc('quarter', generate_series))::INTEGER + 1 AS day_of_quarter
        , EXTRACT(DOY FROM generate_series)::INTEGER AS day_of_year
        , EXTRACT(WEEK FROM generate_series)::INTEGER AS week_of_year
        , EXTRACT(MONTH FROM generate_series)::INTEGER AS month_of_year
        , to_char(generate_series, 'Month') AS month_name
        , to_char(generate_series, 'Mon') AS month_name_abbrev
        , EXTRACT(QUARTER FROM generate_series)::INTEGER AS quarter_of_year
        , EXTRACT(YEAR FROM generate_series)::INTEGER AS year
        , EXTRACT(DAY FROM generate_series + daydiff)::INTEGER AS fiscal_day_of_month
        , DATE_PART('day', generate_series + daydiff - date_trunc('quarter', generate_series + daydiff))::INTEGER + 1 AS fiscal_day_of_quarter
        , EXTRACT(DOY FROM generate_series + daydiff)::INTEGER AS fiscal_day_of_year
        , EXTRACT(WEEK FROM generate_series + daydiff)::INTEGER AS fiscal_week_of_year
        , EXTRACT(MONTH FROM generate_series + daydiff)::INTEGER AS fiscal_month_of_year
        , EXTRACT(QUARTER FROM generate_series + daydiff)::INTEGER AS fiscal_quarter_of_year
        , EXTRACT(YEAR FROM generate_series + daydiff)::INTEGER AS fiscal_year
        , CASE WHEN EXTRACT(DOW FROM generate_series)::INTEGER IN (0, 6) OR h.holiday_id IS NOT NULL THEN FALSE ELSE TRUE END AS is_business_day
        , CASE WHEN EXTRACT(DOW FROM generate_series)::INTEGER IN (0, 6) THEN FALSE ELSE TRUE END AS is_weekday
        , CASE WHEN h.holiday_id IS NULL THEN FALSE ELSE TRUE END AS is_holiday
        , to_char(generate_series, 'YYYYMM') AS YYYYMM
        , to_char(generate_series, 'YYYYMMDD') AS YYYYMMDD
        , to_char(generate_series, 'YYYY0Q') AS YYYYQQ
        , to_char(generate_series, 'YYYY0Q') AS YYYYFQ
    FROM (
        SELECT generate_series(
            startdate::timestamp,
            enddate::timestamp,
            interval '1 days'
        )
    ) a
        LEFT OUTER JOIN master.Holiday h ON a.generate_series = h.observed_date;

    SELECT COUNT(*) INTO record_count FROM master.Calendar;

    RETURN record_count;

END $$ LANGUAGE plpgsql;

Run the stored procedure to create the new master.Calendar table.

In [ ]:
SELECT create_calendar_table(startdate := (date '2017-01-01'), enddate := (date '2017-12-31'), fiscalstartdate := (date '2016-09-01'));